# Open AI Assistants API
The new Assistants API is a stateful evolution of Chat Completions API meant to simplify the creation of assistant-like experiences, and enable developer access to powerful tools like Code Interpreter and Retrieval.
![ALT text](https://raw.githubusercontent.com/panaverse/learn-generative-ai/788f968387b0ad38bc379c3a4718400e8e42948d/03_chatgpt/06_assistants_code_interpreter/assistants.png)

In [16]:
from dotenv import load_dotenv,find_dotenv
import os

load_dotenv(find_dotenv('C:/Code_Apps/Learn-Generative-AI/03_chatgpt/.env'))

api_key = os.environ.get('OPENAI_API_KEY')

In [17]:
from openai import OpenAI
client = OpenAI()

### Code Interpreter
Code Interpreter allows the Assistants API to write and run Python code in a sandboxed execution environment. This tool can process files with diverse data and formatting, and generate files with data and images of graphs. Code Interpreter allows your Assistant to run code iteratively to solve challenging code and math problems. When your Assistant writes code that fails to run, it can iterate on this code by attempting to run different code until the code execution succeeds.<br>
https://platform.openai.com/docs/assistants/tools/code-interpreter

## Step 1: Creating an Assistant

In [35]:
from openai.types.beta.assistant import Assistant
assistant : Assistant = client.beta.assistants.create(
    name="Physics Tutor",
    instructions="You are an expert Physics tutor, having expertise in solving Physics numericals and equations",
    model="gpt-3.5-turbo-1106",
    tools=[{"type":"code_interpreter"}]
)

display(dict(assistant))

{'id': 'asst_FfRNsTJ6GTyo1VnNJEfiJrPg',
 'created_at': 1706730675,
 'description': None,
 'file_ids': [],
 'instructions': 'You are an expert Physics tutor, having expertise in solving Physics numericals and equations',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Physics Tutor',
 'object': 'assistant',
 'tools': [ToolCodeInterpreter(type='code_interpreter')]}

## Step 2: Creating Thread

In [34]:
from openai.types.beta.thread import Thread
thread : Thread = client.beta.threads.create()
display(dict(thread))

{'id': 'thread_fBnISq60zaL0mnFzyROdxz76',
 'created_at': 1706730664,
 'metadata': {},
 'object': 'thread'}

## Step 3: Attaching messages to Thread

In [36]:
from openai.types.beta.threads.thread_message import ThreadMessage

thread_msg : ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the velocity of the car, if it covers 15km in 2 minutes"
)
display(list(thread_msg))

[('id', 'msg_9Dd2W0vxpXkOFrYG5qOfVEDj'),
 ('assistant_id', None),
 ('content',
  [MessageContentText(text=Text(annotations=[], value='What is the velocity of the car, if it covers 15km in 2 minutes'), type='text')]),
 ('created_at', 1706730686),
 ('file_ids', []),
 ('metadata', {}),
 ('object', 'thread.message'),
 ('role', 'user'),
 ('run_id', None),
 ('thread_id', 'thread_fBnISq60zaL0mnFzyROdxz76')]

In [33]:
from openai.types.beta.threads.thread_message import ThreadMessage

thread_msg2 : ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What if it covers 20 km"
)
display(list(thread_msg2))

[('id', 'msg_6r6T7Y9pqJS89H7M3FrNOEvF'),
 ('assistant_id', None),
 ('content',
  [MessageContentText(text=Text(annotations=[], value='What if it covers 20 km'), type='text')]),
 ('created_at', 1706730651),
 ('file_ids', []),
 ('metadata', {}),
 ('object', 'thread.message'),
 ('role', 'user'),
 ('run_id', None),
 ('thread_id', 'thread_eBVKKab3tBbGvn85uNzxpnAR')]

## Step 4: Run the Assistant

In [37]:
from openai.types.beta.threads.run import Run
run_thread : Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please return the values with their SI units"
)
display(dict(run_thread))

{'id': 'run_gAxARdQnaOQJp8EMbYB4gnE2',
 'assistant_id': 'asst_FfRNsTJ6GTyo1VnNJEfiJrPg',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1706730716,
 'expires_at': 1706731316,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'Please return the values with their SI units',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_fBnISq60zaL0mnFzyROdxz76',
 'tools': [ToolAssistantToolsCode(type='code_interpreter')],
 'usage': None}

## Step 5: Check the Run Status

In [38]:
run : Run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id =    run_thread.id
)
display(dict(run))

{'id': 'run_gAxARdQnaOQJp8EMbYB4gnE2',
 'assistant_id': 'asst_FfRNsTJ6GTyo1VnNJEfiJrPg',
 'cancelled_at': None,
 'completed_at': 1706730721,
 'created_at': 1706730716,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'Please return the values with their SI units',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1706730716,
 'status': 'completed',
 'thread_id': 'thread_fBnISq60zaL0mnFzyROdxz76',
 'tools': [ToolAssistantToolsCode(type='code_interpreter')],
 'usage': Usage(completion_tokens=58, prompt_tokens=303, total_tokens=361)}

## Step 6: Display the Assistant Response

In [31]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
)

# reversed(list) -> Does not modify original list
# list.reverse() -> Modifies original list
print('Messages: ',messages)
print()
print()

for msg in reversed(messages.data):
    print(f"{msg.role}: {msg.content[0].text.value}")

Messages:  SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_fsvtFnf1qOqazRZusFIg7rkW', assistant_id='asst_5zu2j2lfiqJJceErpOn00ovj', content=[MessageContentText(text=Text(annotations=[], value='If the car covers 20 km in 2 minutes, the velocity will be \\(166.67 \\, m/s\\)'), type='text')], created_at=1706730380, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_0iaXztybuwQ3C8POg2HDb6rX', thread_id='thread_eBVKKab3tBbGvn85uNzxpnAR'), ThreadMessage(id='msg_W9Aow2UaBKmevasSD0zOnOwr', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What if it covers 20 km'), type='text')], created_at=1706730369, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_eBVKKab3tBbGvn85uNzxpnAR'), ThreadMessage(id='msg_USFGSlU4RG3sTn4M8IpUElUa', assistant_id='asst_5zu2j2lfiqJJceErpOn00ovj', content=[MessageContentText(text=Text(annotations=[], value='The velocity of the car is \\(125 \\, m/s\\)'), t